In [1]:
import gc
import os
import random
import warnings
import numpy as np
import pandas as pd
from IPython.display import display
import timm
import torch
import torch.nn as nn  
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchvision.transforms as transforms
from tqdm import tqdm

warnings.filterwarnings('ignore', category=Warning)
gc.collect()

36

In [2]:
class Config:
    seed = 42 
    image_transform = transforms.Resize((512,512))  
    batch_size = 32
    num_epochs = 9
    num_folds = 5

def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
set_seed(Config.seed)

def KL_loss(p,q):
    epsilon=10**(-15)
    p=torch.clip(p,epsilon,1-epsilon)
    q = nn.functional.log_softmax(q,dim=1)
    return torch.mean(torch.sum(p*(torch.log(p)-q),dim=1))

gc.collect()

0

In [3]:
train_df = pd.read_csv("updated_train.csv")

labels = ['seizure', 'lpd', 'gpd', 'lrda', 'grda', 'other']

train_feats = pd.DataFrame()

for label in labels:    group = train_df[f'{label}_vote'].groupby(train_df['spectrogram_id']).sum()
    label_vote_sum = pd.DataFrame({'spectrogram_id': group.index, f'{label}_vote_sum': group.values})
    if label == 'seizure':
        train_feats = label_vote_sum
    else:
        train_feats = train_feats.merge(label_vote_sum, on='spectrogram_id', how='left')

train_feats['total_vote'] = 0
for label in labels:
    train_feats['total_vote'] += train_feats[f'{label}_vote_sum']

for label in labels:
    train_feats[f'{label}_vote'] = train_feats[f'{label}_vote_sum'] / train_feats['total_vote']

choose_cols = ['spectrogram_id']
for label in labels:
    choose_cols += [f'{label}_vote']
train_feats = train_feats[choose_cols]

train_feats['path'] = train_feats['spectrogram_id'].apply(lambda x: "E:/HMS2024/train_spec_for_paper/" + str(x) + ".parquet")

gc.collect()

0

In [4]:
def get_batch(paths, batch_size=Config.batch_size):
    eps = 1e-6
    batch_data = []

    for path in paths:
        data = pd.read_parquet(path[0])
        data = data.fillna(-1).values[:, 1:].T
        data = np.clip(data, np.exp(-6), np.exp(10))
        data = np.log(data)
        data_mean = data.mean(axis=(0, 1))
        data_std = data.std(axis=(0, 1))
        data = (data - data_mean) / (data_std + eps)
        data_tensor = torch.unsqueeze(torch.Tensor(data), dim=0)
        data = Config.image_transform(data_tensor)
        batch_data.append(data)

    batch_data = torch.stack(batch_data)
    return batch_data

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(f"Using device: {device}")

total_idx = np.arange(len(train_feats))
np.random.shuffle(total_idx)

gc.collect()

for fold in range(Config.num_folds):
    test_idx = total_idx[fold * len(total_idx) // Config.num_folds:(fold + 1) * len(total_idx) // Config.num_folds]
    train_idx = np.array([idx for idx in total_idx if idx not in test_idx])
    model = timm.create_model('tf_efficientnet_b0_ns', pretrained=True, num_classes=6, in_chans=1)
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=0.001, betas=(0.5, 0.999), weight_decay=0.01)
    scheduler = CosineAnnealingLR(optimizer, T_max=Config.num_epochs)
    best_test_loss = float('inf')
    train_losses = []
    test_losses = []
    
    print(f"Starting training for fold {fold + 1}")

    for epoch in range(Config.num_epochs):
        model.train()
        train_loss = []
        random_num = np.arange(len(train_idx))
        np.random.shuffle(random_num)
        train_idx = train_idx[random_num]

        for idx in tqdm(range(0, len(train_idx), Config.batch_size)):
            optimizer.zero_grad()
            train_idx1 = train_idx[idx:idx + Config.batch_size]
            train_X1_path = train_feats[['path']].iloc[train_idx1].values
            train_X1 = get_batch(train_X1_path, batch_size=Config.batch_size)
            train_y1 = train_feats[['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']].iloc[train_idx1].values
            train_y1 = torch.Tensor(train_y1)

            train_pred = model(train_X1.to(device))
            loss = KL_loss(train_y1.to(device), train_pred)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())

        epoch_train_loss = np.mean(train_loss)
        train_losses.append(epoch_train_loss)
        print(f"Epoch {epoch + 1}: Train Loss = {epoch_train_loss:.2f}")

        scheduler.step()

        model.eval()
        test_loss = []
        with torch.no_grad():
            for idx in tqdm(range(0, len(test_idx), Config.batch_size)):
                test_idx1 = test_idx[idx:idx + Config.batch_size]
                test_X1_path = train_feats[['path']].iloc[test_idx1].values
                test_X1 = get_batch(test_X1_path, batch_size=Config.batch_size)
                test_y1 = train_feats[['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']].iloc[test_idx1].values
                test_y1 = torch.Tensor(test_y1)

                test_pred = model(test_X1.to(device))
                loss = KL_loss(test_y1.to(device), test_pred)
                test_loss.append(loss.item())

        epoch_test_loss = np.mean(test_loss)
        test_losses.append(epoch_test_loss)
        print(f"Epoch {epoch + 1}: Test Loss = {epoch_test_loss:.2f}")

        if epoch_test_loss < best_test_loss:
            best_test_loss = epoch_test_loss
            torch.save(model.state_dict(), f"efficientnet_b0_fold{fold}.pth")

        gc.collect()

    print(f"Fold {fold + 1} Best Test Loss: {best_test_loss:.2f}")

Using device: cuda
Starting training for fold 1


100%|████████████████████████████████████████████████████████████████████████████████| 261/261 [07:31<00:00,  1.73s/it]


Epoch 1: Train Loss = 1.36


100%|██████████████████████████████████████████████████████████████████████████████████| 66/66 [01:23<00:00,  1.26s/it]


Epoch 1: Test Loss = 1.29


...

Epoch 8: Test Loss = 1.22


100%|████████████████████████████████████████████████████████████████████████████████| 261/261 [06:43<00:00,  1.54s/it]


Epoch 9: Train Loss = 1.22


100%|██████████████████████████████████████████████████████████████████████████████████| 66/66 [01:19<00:00,  1.21s/it]

Epoch 9: Test Loss = 1.21
Fold 5 Best Test Loss: 1.21
